In [1]:
import ast
import copy
import glob
import json
import os
import re
import shutil

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from IPython.core.interactiveshell import InteractiveShell
from PIL import Image
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from tqdm import tqdm

InteractiveShell.ast_node_interactivity = "all"
import copy

import seaborn as sns
import torch
import torchvision
from IPython.display import clear_output
from torchvision.ops import box_iou

In [2]:
TRAIN_DF_PART = "/app/_data/tensorflow-great-barrier-reef/train.csv"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE
SEED = 37
IMAGE_FOLDER = "images"
LABEL_FOLDER = "labels"

'cuda'

In [3]:
df = pd.read_csv(TRAIN_DF_PART)
df["img_path"] = (
    "/app/_data/tensorflow-great-barrier-reef/train_images/video_"
    + df.video_id.astype("str")
    + "/"
    + df.video_frame.astype("str")
    + ".jpg"
)
df["annotations"] = df["annotations"].apply(lambda x: ast.literal_eval(x))
df["len_annotation"] = df["annotations"].str.len()
df["image_id"] = df["image_id"].str.replace("-", "_", regex=True)
df["new_img_path"] = f"/app/_data/{IMAGE_FOLDER}/" + df["image_id"] + ".jpg"
df["label"] = df["len_annotation"].apply(lambda x: 0 if x == 0 else 1)
df["no_label"] = df["len_annotation"].apply(lambda x: True if x == 0 else False)
R = df[df["len_annotation"] == 0].shape[0] / df[df["len_annotation"] != 0].shape[0]
df["label_change"] = df["label"] & df["no_label"].shift(1) & df["no_label"].shift(
    2
) | df["no_label"] & df["label"].shift(1) & df["label"].shift(2)
df["sequense_change"] = df["sequence"] != df["sequence"].shift(1)
df["start_subseq"] = df["sequense_change"] | df["label_change"]
df.loc[df.index[-1], "start_subseq"] = True
df["start_subseq"].sum()
start_idx = 0
for subsequence_id, end_idx in enumerate(df[df["start_subseq"]].index):
    df.loc[start_idx:end_idx, "subsequence_id"] = subsequence_id
    start_idx = end_idx

df["subsequence_id"] = df["subsequence_id"].astype(int)
df["subsequence_id"].nunique()

138

137

In [4]:
df_split = (
    df.groupby("subsequence_id")
    .agg({"label": "max", "len_annotation": "sum", "video_frame": "count"})
    .astype(int)
    .reset_index()
)
n_splits = 10
y = df_split["label"]
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)

for fold_id, (train_idx, val_idx) in enumerate(
    skf.split(df_split["subsequence_id"], y=y)
):
    subseq_val_idx = df_split["subsequence_id"].iloc[val_idx]
    df.loc[df["subsequence_id"].isin(subseq_val_idx), "fold_label"] = fold_id
df["fold_label"] = df["fold_label"].astype(int)

y = df_split["len_annotation"]

for fold_id, (train_idx, val_idx) in enumerate(
    skf.split(df_split["subsequence_id"], y=y)
):
    subseq_val_idx = df_split["subsequence_id"].iloc[val_idx]
    df.loc[df["subsequence_id"].isin(subseq_val_idx), "fold_ann"] = fold_id

df["fold_ann"] = df["fold_ann"].astype(int)

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


In [9]:
with open("/app/_data/sequences.json", "r") as f:
    seq_dict = json.load(f)
with open("/app/f2_results.json", "r") as f:
    res_dict = json.load(f)

In [10]:
def tp_fp_fn(gt, prediction, conf_thr):
    ious = np.arange(0.3, 0.81, 0.05)
    TP, FP, FN = (
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
        np.zeros(ious.shape[0], "int16"),
    )
    prediction = prediction[prediction[:, 4] > conf_thr]
    bboxes = prediction[:, :4].astype("int")
    bboxes[:, 0] = bboxes[:, 0] - bboxes[:, 2] / 2
    bboxes[:, 1] = bboxes[:, 1] - bboxes[:, 3] / 2
    bboxes[:, 2] = bboxes[:, 0] + bboxes[:, 2]
    bboxes[:, 3] = bboxes[:, 1] + bboxes[:, 3]
    if bboxes.size != 0:
        if gt.size == 0:
            fp = bboxes.shape[0]
            FP = np.full(ious.shape[0], fp, "int16")
        else:
            iou_matrix = box_iou(torch.Tensor(gt), torch.Tensor(bboxes))
            for n, iou_thr in enumerate(ious):
                x = torch.where(iou_matrix >= iou_thr)
                tp = np.unique(x[0]).shape[0]
                fp = bboxes.shape[0] - tp
                fn = gt.shape[0] - tp
                TP[n] = tp
                FP[n] = fp
                FN[n] = fn
    else:
        if gt.size != 0:
            fn = gt.shape[0]
            FN = np.full(ious.shape[0], fn, "int16")
    return TP, FP, FN

## KFold split

In [11]:
paths = glob.glob("/app/_data/*/runs/train/*/*/*.pt")
paths = [k for k in paths if k not in res_dict]
paths = ['/app/_data/yolov5_f2/runs/train/yolov5l6_2560_subseq_3_f2_rect_005_resume/weights/best.pt']

In [12]:
R = df[df["len_annotation"] == 0].shape[0] / df[df["len_annotation"] != 0].shape[0]
conf_thres = np.arange(0.1, 0.61, 0.01)
ious = np.arange(0.3, 0.81, 0.05)

In [17]:
res = np.zeros([conf_thres.shape[0], 3, ious.shape[0]])

for a in range(len(paths)):
    if paths[a] not in res_dict.keys():
        # load model
        path = paths[a]
        IMG_SIZE = 2560 if '2560' in path else 3008 if "3008" in path else 2880 if "2880" in path else 2880
        model = torch.hub.load(
            "/app/_data/yolov5", "custom", path=path, source="local", force_reload=True
        )
        model.conf = 0.01
        # chose validation set
        if "val" in path:
            VIDEO_ID = path[path.index("val") + 3]
            if VIDEO_ID == "a" or VIDEO_ID == "8":
                VIDEO_ID = 2
            else:
                VIDEO_ID = int(VIDEO_ID)
            if VIDEO_ID == 3:
                VIDEO_ID = 0
            df_test = df.query("video_id==@VIDEO_ID").reset_index(drop=True)
        elif "_subseq_9" in path:
            df_test = df.query('fold_ann==9').reset_index(drop=True)
        elif "subseq_3" in path:
            df_test = df.query('fold_label==3').reset_index(drop=True)
            if path[path.index("seq_3008_") + 10] != "_":
                val_seq = path[
                    path.index("seq_3008_") + 9 : path.index("seq_3008_") + 11
                ]
            else:
                val_seq = path[path.index("seq_3008_") + 9]
            val = seq_dict[val_seq]["val"]
            df_test = (
                pd.concat(
                    [
                        df.query("sequence in @val and len_annotation!=0"),
                        df.query("sequence in @val and len_annotation==0").sample(
                            int(
                                R
                                * df.query(
                                    "sequence in @val and len_annotation!=0"
                                ).shape[0]
                            )
                        ),
                    ],
                    ignore_index=True,
                )
                .sample(frac=1)
                .reset_index(drop=True)
            )
#         computing f2 score
        for ix in tqdm(df_test.index.tolist()):
            img = np.array(Image.open(df_test.loc[ix, "img_path"]))
            prediction = model(img, size=IMG_SIZE, augment=True).xywh[0].cpu().numpy()
            prediction = prediction[prediction[:, 4] > 0.1]
            gt = np.array([list(x.values()) for x in df_test.loc[ix, "annotations"]])
            if gt.size:
                gt[:, 2] = gt[:, 2] + gt[:, 0]
                gt[:, 3] = gt[:, 3] + gt[:, 1]
            for n, c_th in enumerate(conf_thres):
                TP, FP, FN = tp_fp_fn(gt, prediction, c_th)
                res[n, 0, :] += TP
                res[n, 1, :] += FP
                res[n, 2, :] += FN
        F2 = np.zeros(conf_thres.shape[0])
        for c in range(conf_thres.shape[0]):
            TP = res[c, 0, :]
            FP = res[c, 1, :]
            FN = res[c, 2, :]
            recall = TP / (TP + FN)
            precission = TP / (TP + FP)
            f2 = 5 * precission * recall / (4 * precission + recall + 1e-16)
            F2[c] = np.mean(f2)
        if path not in res_dict:
            res_dict[path] = {
                IMG_SIZE: {
                    "best": [
                        np.round(conf_thres[np.argmax(F2)], 2),
                        np.round(np.max(F2), 4),
                    ],
                    "all": list(np.round(F2, 4)),
                }
            }
        else:
            res_dict[path][IMG_SIZE] = {
                "best": [
                    np.round(conf_thres[np.argmax(F2)], 2),
                    np.round(np.max(F2), 4),
                ],
                "all": list(np.round(F2, 4)),
            }
        with open("/app/f2_results.json", "w") as f:
            json.dump(res_dict, f)

YOLOv5 🚀 v6.0-193-gdb1f83b torch 1.9.1+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24268MiB)

Fusing layers... 
Model Summary: 476 layers, 76118664 parameters, 0 gradients
Adding AutoShape... 
100% 3716/3716 [11:59<00:00,  5.16it/s]


In [18]:
res_dict[path]

{'2560': {'best': [0.43, 0.8484],
  'all': [0.8143,
   0.8184,
   0.8226,
   0.8268,
   0.8302,
   0.832,
   0.8317,
   0.8337,
   0.8357,
   0.836,
   0.8373,
   0.8379,
   0.8385,
   0.8404,
   0.8417,
   0.8423,
   0.843,
   0.8429,
   0.8439,
   0.8428,
   0.8429,
   0.8435,
   0.8438,
   0.8458,
   0.8461,
   0.8467,
   0.8471,
   0.8461,
   0.8452,
   0.8458,
   0.8468,
   0.8471,
   0.8471,
   0.8484,
   0.8484,
   0.8468,
   0.8468,
   0.8459,
   0.8446,
   0.8446,
   0.8449,
   0.8452,
   0.8438,
   0.8438,
   0.8428,
   0.8432,
   0.8438,
   0.8441,
   0.8392,
   0.8318,
   0.8275]},
 2560: {'best': [0.1, 0.4457],
  'all': [0.4457,
   0.4448,
   0.4431,
   0.4417,
   0.4415,
   0.4413,
   0.4407,
   0.4385,
   0.4357,
   0.4345,
   0.4345,
   0.4306,
   0.4281,
   0.4273,
   0.426,
   0.4231,
   0.4224,
   0.4202,
   0.4188,
   0.4167,
   0.4158,
   0.4135,
   0.4125,
   0.4101,
   0.4099,
   0.4087,
   0.4058,
   0.4042,
   0.4038,
   0.4008,
   0.3986,
   0.3971,
   0.3953,